In [11]:
import json
from collections import defaultdict
from sklearn import linear_model, model_selection
import numpy as np
import random
import gzip
import dateutil.parser
import math
import mido
import pretty_midi
import joblib
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec
import collections
import os
import tarfile
import tables
import hdf5_getters as GETTERS
from midi_methods import *
import h5py

# %store -r

In [2]:

# # UNCOMMENT AND RUN THIS BLOCK OF CODE IF YOU HAVE NOT EXTRACTED MIDI FILES
# # FROM GZ FILE

# # Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_aligned.tar.gz
# # Put it in the resources folder!!!

# f = tarfile.open("../sandbox/lmd_matched.tar.gz")
# f.extractall()
# f.close

# # Download following file at http://hog.ee.columbia.edu/craffel/lmd/lmd_matched_h5.tar.gz
# # Put it in the resources folder!!!

# f = tarfile.open("../sandbox/lmd_matched_h5.tar.gz")
# f.extractall()
# f.close

In [3]:
# https://github.com/craffel/pretty-midi/blob/main/Tutorial.ipynb

# Some song that was extracted
test_song_path = '../lmd_unwrapped_files/lmd_matched/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid'

song_pm = pretty_midi.PrettyMIDI(test_song_path)
all_instruments = song_pm.instruments
# Instrument.program to get program number
all_instruments[:5]

[Instrument(program=62, is_drum=False, name="main synth          "),
 Instrument(program=62, is_drum=False, name="echo synth          "),
 Instrument(program=48, is_drum=False, name="main strings h      "),
 Instrument(program=48, is_drum=False, name="main strings m      "),
 Instrument(program=48, is_drum=False, name="main strings l      ")]

In [4]:
song_pm.instruments

[Instrument(program=62, is_drum=False, name="main synth          "),
 Instrument(program=62, is_drum=False, name="echo synth          "),
 Instrument(program=48, is_drum=False, name="main strings h      "),
 Instrument(program=48, is_drum=False, name="main strings m      "),
 Instrument(program=48, is_drum=False, name="main strings l      "),
 Instrument(program=33, is_drum=False, name="main bass           "),
 Instrument(program=0, is_drum=True, name="drums + fills       "),
 Instrument(program=14, is_drum=False, name="xtras               "),
 Instrument(program=6, is_drum=False, name="harpsichord         "),
 Instrument(program=19, is_drum=False, name="verse vibe h        "),
 Instrument(program=19, is_drum=False, name="verse vibe m        "),
 Instrument(program=19, is_drum=False, name="verse vibe l        "),
 Instrument(program=1, is_drum=False, name="verse piano         "),
 Instrument(program=64, is_drum=False, name="verse xtra          "),
 Instrument(program=48, is_drum=False,

In [5]:
# FIrst instrument listed is main synth. Name isnt as important as program number
# as name can be assigned to anything.
main_synth = all_instruments[0]
main_synth

Instrument(program=62, is_drum=False, name="main synth          ")

In [6]:
# Get specific notes played for each instrument
notes_main_synth = main_synth.notes
notes_main_synth[0], notes_main_synth[-1] 

(Note(start=0.018229, end=0.184896, pitch=69, velocity=100),
 Note(start=215.851562, end=215.989583, pitch=71, velocity=100))

In [64]:
# Respective song's hdf5 file

test_hdf5_path = '../lmd_unwrapped_files/lmd_matched_h5\A\A\A\TRAAAGR128F425B14B.h5'
h5 = GETTERS.open_h5_file_read(test_hdf5_path)
# Get song metadata
h5.root.metadata    

array([b'classic pop and rock'], dtype='|S256')

In [61]:
h5.root.metadata.songs.cols.title[:][0].decode('utf-8') 

'Into The Nightlife'

In [12]:
# Get song title
h5.root.metadata.songs.cols.title[:]

array([b'Into The Nightlife'], dtype='|S1024')

In [71]:
# Get album

h5.root.metadata.songs.cols.release[:]

array([b'Bring Ya To The Brink'], dtype='|S1024')

In [14]:
# Get song id. I don't know what the id is referring to (just that dataset or 
# also a foreign key to another datset)

h5.root.metadata.songs.cols.song_id[:]

array([b'SONRWUU12AF72A4283'], dtype='|S32')

In [15]:
# Get song artist

h5.root.metadata.songs.cols.artist_name[:]

array([b'Cyndi Lauper'], dtype='|S1024')

In [16]:
# Getting genre of a song. If its empty, that means it has no genre assigned in 
# the dataset, but may still have a genre
h5.root.metadata.songs.cols.genre[:]

array([b''], dtype='|S1024')

In [70]:
# get tag info : lmd_matched_h5 has data associated with MB metadata
h5.root.musicbrainz.artist_mbtags[:]

array([b'classic pop and rock'], dtype='|S256')

In [17]:
#https://github.com/tbertinmahieux/MSongsDB/blob/master/PythonSrc/hdf5_getters.py